In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os, sys, time
import gdal
import datetime
import fnmatch
import math
import pandas as pd
from glob import glob

%matplotlib inline

In [2]:
year = 2016
sif_file = 'sif/mcd43_b7_sza_vpd_temp_sif_training.nc4'
avhrr_path = 'www.ncei.noaa.gov/data/avhrr-land-surface-reflectance/access'
avhrr_path = avhrr_path + '/' + str(year) + '/'
prefix ='AVHRR-Land_v005_AVH09C1_NOAA-19_'

In [3]:
yaos_sif = xr.open_dataset(sif_file)

In [4]:
sif = yaos_sif.copy()

In [5]:
df = pd.DataFrame(sif.yday.values, columns = ['yday']) 
df['lat'] = sif.lat.values.round(3)
df['lon'] = sif.lon.values.round(3)
df['sif'] = sif.sif_757nm.values
df['b1'] = sif.b1.values
df['b2'] = sif.b2.values
df['sza'] = sif.sza.values

df['doy'] = sif.yday.values - year*1000
df['doy'] = df['doy'] - 1 
df['date'] = pd.to_datetime(df.doy,unit='D',origin=str(year))
#df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
#df['day'] = pd.DatetimeIndex(df['date']).day
# df[df.yday == 2015365]

df['lon'] = df['lon'].round(decimals=3)
df['lat'] = df['lat'].round(decimals=3)

In [6]:
df_new = df.where(df.year == year)
df_new = df_new.dropna(axis=0)
df_new['lon'] = df_new['lon'].round(decimals=3)
df_new['lat'] = df_new['lat'].round(decimals=3)

df_new = df_new.drop(columns=['doy','year'])

In [7]:
final_df = df_new.copy()

In [19]:
#year = 2016
#i= 201 # df_new.doy = 13
#date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
#date

datetime.datetime(2016, 7, 19, 0, 0)

In [8]:
for i in range(1,2):
    print(i)
    # i= 14 # df_new.doy = 13
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
    avhrr_file = avhrr_path + prefix + format(int(year), '04d') + format(int(date.month), '02d') + format(int(date.day), '02d') + '_*.nc'
    avhrr = xr.open_dataset(glob(avhrr_file)[0])
    avhrr = avhrr.sel(latitude = slice(45,-40))
    
    avhrr_red = pd.DataFrame(avhrr.SREFL_CH1.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_red.index.name = 'lat'
    avhrr_red.columns.name = 'lon'
    avhrr_red = avhrr_red.stack(dropna=True)
    avhrr_red.name = 'red'
    avhrr_red = avhrr_red.reset_index()
    avhrr_red['date'] = date
    avhrr_red['lon'] = avhrr_red['lon'].round(3)
    avhrr_red['lat'] = avhrr_red['lat'].round(3)
    
    avhrr_nir = pd.DataFrame(avhrr.SREFL_CH2.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_nir.index.name = 'lat'
    avhrr_nir.columns.name = 'lon'
    avhrr_nir = avhrr_nir.stack(dropna=True)
    avhrr_nir.name = 'nir'
    avhrr_nir = avhrr_nir.reset_index()
    avhrr_nir['lon'] = avhrr_nir['lon'].round(3)
    avhrr_nir['lat'] = avhrr_nir['lat'].round(3)
    
    avhrr_szen = pd.DataFrame(avhrr.SZEN.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_szen.index.name = 'lat'
    avhrr_szen.columns.name = 'lon'
    avhrr_szen = avhrr_szen.stack(dropna=True)
    avhrr_szen.name = 'szen'
    avhrr_szen = avhrr_szen.reset_index()
    avhrr_szen['lon'] = avhrr_szen['lon'].round(3)
    avhrr_szen['lat'] = avhrr_szen['lat'].round(3)
    
    avhrr_df = pd.merge(avhrr_nir, avhrr_red, on=['lat','lon'])    
    avhrr_df = pd.merge(avhrr_df, avhrr_szen, on=['lat','lon'])
    final_df = pd.merge(avhrr_df, final_df, on=['lat','lon','date'])
    final_df.to_csv('out_2016.csv', index=False)

1


In [28]:
### if the kernel breaks down, read the file to load back the data
### ignore above one cell and start from here
#final_df = pd.read_csv('out_2016.csv', index_col=False) 
#final_df

In [9]:
for i in range(2,367):
    print(i)
    # i= 14 # df_new.doy = 13
    tmp_df = df_new.copy()
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(i - 1)
    avhrr_file = avhrr_path + prefix + format(int(year), '04d') + format(int(date.month), '02d') + format(int(date.day), '02d') + '_*.nc'
    #print(avhrr_file)
    avhrr = xr.open_dataset(glob(avhrr_file)[0])
    avhrr = avhrr.sel(latitude = slice(45,-40))
    
    avhrr_red = pd.DataFrame(avhrr.SREFL_CH1.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_red.index.name = 'lat'
    avhrr_red.columns.name = 'lon'
    avhrr_red = avhrr_red.stack(dropna=True)
    avhrr_red.name = 'red'
    avhrr_red = avhrr_red.reset_index()
    avhrr_red['date'] = date
    avhrr_red['lon'] = avhrr_red['lon'].round(3)
    avhrr_red['lat'] = avhrr_red['lat'].round(3)
    
    avhrr_nir = pd.DataFrame(avhrr.SREFL_CH2.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_nir.index.name = 'lat'
    avhrr_nir.columns.name = 'lon'
    avhrr_nir = avhrr_nir.stack(dropna=True)
    avhrr_nir.name = 'nir'
    avhrr_nir = avhrr_nir.reset_index()
    avhrr_nir['lon'] = avhrr_nir['lon'].round(3)
    avhrr_nir['lat'] = avhrr_nir['lat'].round(3)
    
    avhrr_szen = pd.DataFrame(avhrr.SZEN.values[0], index= avhrr.latitude.values, columns= avhrr.longitude.values) 
    avhrr_szen.index.name = 'lat'
    avhrr_szen.columns.name = 'lon'
    avhrr_szen = avhrr_szen.stack(dropna=True)
    avhrr_szen.name = 'szen'
    avhrr_szen = avhrr_szen.reset_index()
    avhrr_szen['lon'] = avhrr_szen['lon'].round(3)
    avhrr_szen['lat'] = avhrr_szen['lat'].round(3)
    
    avhrr_df = pd.merge(avhrr_nir, avhrr_red, on=['lat','lon'], sort=False)    
    avhrr_df = pd.merge(avhrr_df, avhrr_szen, on=['lat','lon'], sort=False)
    tmp_df = pd.merge(avhrr_df, tmp_df, on=['lat','lon','date'], sort=False)
    final_df = final_df.append(tmp_df, ignore_index=True, sort=False)
    final_df.to_csv('out_2016.csv', index=False)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [ ]:
final_df.plot(kind='scatter',x='red',y='sif')

In [ ]:
final_df.plot(kind='scatter',x='b1',y='sif')

In [ ]:
final_df.plot(kind='scatter',x='b1',y='red')

In [ ]:
num_points = len(final_df.index)
num_points